# Exploratory Data Analysis
 
 Following scraping data, we clean and explore the relationships between variables

## Cleaning the data

In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [21]:
df=pd.read_pickle('./FullVGChartzDatabase.zip',compression = 'zip')

In [22]:
# Turn all N/A's into np.nan values so pandas can operate with them easily
df_obj = df.select_dtypes('object')
df[df.columns] = df_obj.apply(lambda x: x.str.strip())
df=df.replace('N/A',np.nan)

In [26]:
# Merge columns total sales and total shipped because they describe the same thing but one is \
# for games no longer in production
df['total_shipped'].fillna(df['total_sales'],inplace = True)
df.dropna(subset=['total_shipped'],inplace = True)
df.drop(columns = ['total_sales'],inplace = True)

In [27]:
df

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,Wii Sports,Wii,Nintendo,Nintendo EAD,NaN,7.7,NaN,82.86m,NaN,NaN,NaN,NaN,19th Nov 06,NaN
1,2,Super Mario Bros.,NES,Nintendo,Nintendo EAD,NaN,10.0,8.2,40.24m,NaN,NaN,NaN,NaN,18th Oct 85,NaN
2,3,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,8.7,8.2,9.1,37.14m,NaN,NaN,NaN,NaN,27th Apr 08,11th Apr 18
3,4,PlayerUnknown's Battlegrounds,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,NaN,36.60m,NaN,NaN,NaN,NaN,21st Dec 17,13th Nov 18
4,5,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,8.8,8.0,8.8,33.09m,NaN,NaN,NaN,NaN,26th Jul 09,NaN
5,6,Pokémon Red / Green / Blue Version,GB,Nintendo,Game Freak,NaN,9.4,NaN,31.38m,NaN,NaN,NaN,NaN,30th Sep 98,NaN
6,7,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,NaN,9.1,8.1,30.80m,NaN,NaN,NaN,NaN,15th May 06,NaN
7,8,Tetris,GB,Nintendo,Bullet Proof Software,NaN,NaN,NaN,30.26m,NaN,NaN,NaN,NaN,01st Jun 89,NaN
8,9,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,9.1,8.6,9.2,30.22m,NaN,NaN,NaN,NaN,15th Nov 09,NaN
9,10,Minecraft,PC,Mojang,Mojang AB,NaN,10.0,NaN,30.01m,NaN,NaN,NaN,NaN,10th May 10,05th Aug 18


In [18]:
def remove_last_char(string):
    string = string[:,-1]
    return string

In [33]:
df[['total_shipped','na_sales']]

,total_shipped,na_sales
0,82.86m,NaN
1,40.24m,NaN
2,37.14m,NaN
3,36.60m,NaN
4,33.09m,NaN
5,31.38m,NaN
6,30.80m,NaN
7,30.26m,NaN
8,30.22m,NaN
9,30.01m,NaN


In [57]:
df.dtypes

position         object
game             object
console          object
publisher        object
developer        object
vgchart_score    object
critic_score     object
user_score       object
total_shipped    object
na_sales         object
pal_sales        object
japan_sales      object
other_sales      object
release_date     object
last_update      object
dtype: object

In [62]:
df[['total_shipped','na_sales','pal_sales','japan_sales','other_sales']].apply(pd.DataFrame.to_string(str))

TypeError: astype() missing 1 required positional argument: 'dtype'

In [48]:
#.apply(lambda str_val: str(str_val)[:-1])

'tes'

In [59]:
test

total_shipped    0        82.86m\n1        40.24m\n2        37....
na_sales         0          nan\n1          nan\n2          nan...
pal_sales        0          nan\n1          nan\n2          nan...
japan_sales      0          nan\n1          nan\n2          nan...
other_sales      0          nan\n1          nan\n2          nan...
dtype: object